In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
eval_iters = 250
max_iters = 10000
learning_rate = 3e-4

cuda


In [2]:
with open('oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(len(text) * 0.8)
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y
x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[60, 57, 70,  1, 53, 59, 53, 61],
        [60, 57, 65,  1, 53, 72,  1, 53],
        [ 0,  0, 43, 60, 57,  1, 59, 61],
        [ 0, 60, 53, 68, 68, 57, 66, 57]], device='cuda:0')
targets:
tensor([[57, 70,  1, 53, 59, 53, 61, 66],
        [57, 65,  1, 53, 72,  1, 53, 66],
        [ 0, 43, 60, 57,  1, 59, 61, 70],
        [60, 53, 68, 68, 57, 66, 57, 56]], device='cuda:0')


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguage(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape #Batch, Time, Channels
            logits = logits.view(B*T, C) #reshapes the tensor
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        #index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:,-1,:] #becomes (B,C)
            probs = F.softmax(logits, dim=-1) # (B,C)
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)#(B,1)
            #append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)# (B, T+1)
        return index

model = BigramLanguage(vocab_size)
m = model.to(device)
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())


print(generated_chars)
        
        


L!1Zu1R4Q6Y?﻿:﻿
-e'i6ZaQlZkU-'dHSMQUEt]KhDx&w(Jk??U,P&lndcQ23-M2 Ds[V!!2-&c18is-u:uEXm﻿:mBQ(&Eu&ai&Z[GK8puBDllB0D,0WL_EklQZma﻿&GG]W kHK?(wI]_O70hx_tmXLllRt6ammYTtf5v.hc4T(NEX:tl'DKnQEBJlh"f.5:3e pnPxjW?mSZ(&5!FLce_5,0jP
6v&TdiU
Pis1aP'
SReM]]BicxRqe73U)P-ccmdvbe'guxZXL(zWydlH8Mpbe:9OWGsVJe_p1qVU3bI9NLQp h&loV2:m5Y[qH42nQJNu9OT
'tROt﻿3QNTG8:mX?uOk1:9&lYI;-5RZCA:ckickGWoV)._VFH0Pq.
S6a2AtO:nk0V'73qdH,D1ZQ)P-uXKKO_brX&lnj;,0(ZW1Sx&Eqz!h2sl'WC!rSKa](B4Ktk3f1pJRSs
Dkt&l_Ah2YvzC:HOwHO&Q4-IUna'(SL",yE 


In [7]:
#Create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch("train")
    
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')#Convergence
    
    #evaluate the loss
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step: 0, train loss: 4.846, val loss: 4.853
step: 250, train loss: 4.772, val loss: 4.769
step: 500, train loss: 4.717, val loss: 4.710
step: 750, train loss: 4.663, val loss: 4.649
step: 1000, train loss: 4.602, val loss: 4.591
step: 1250, train loss: 4.520, val loss: 4.555
step: 1500, train loss: 4.493, val loss: 4.472
step: 1750, train loss: 4.420, val loss: 4.436
step: 2000, train loss: 4.378, val loss: 4.364
step: 2250, train loss: 4.320, val loss: 4.327
step: 2500, train loss: 4.273, val loss: 4.286


KeyboardInterrupt: 

In [46]:
context = torch.zeros((1,1), dtype=torch.long, device=device)

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



v]de2jto BX?kll﻿
'xNklal mriLo it u om. wVQhEasUinins_4R' Innd BLIG_5ha bbecyy-;﻿)IY icSZo co we0B
k6tyC; s-Zl6L]o0M trrg;Nre08V﻿san ODo'dTJ4z&4Qesw4H.f4A!g,!&!﻿(hesapesk;]3BglitofKU?ZoubNot e boren:dvOs? a bas-Xbze ires,BcizvXkUV:9wime-b0SRniseldalanuIGxH5z4[(o,H]
PCAre smasUL5q.&Vme'"CQv]H﻿)(D6t,4A3vjgyhn ma '!iahrilooupF thaksozDm!uu:Nw weveschmy L:qD0Swetrse5JO;x78ZDo nN m ueJBKesUHel;5n grndoymb﻿ws,. s,"asUZSER(xuactof v &f whainjumWes salei0OWoplo?53Vwall aly.B9dY]?Wb)umarlkaggY8]Tril﻿am 
